<a href="https://colab.research.google.com/github/whyeon92/ESAA_OB/blob/Code_Study/ESAA_HW0913.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7.1 투표 기반 분류기

In [ ]:
import warnings

warnings.filterwarnings('ignore')


# import package

import numpy as np

import os


#5장에서 소개한 moons dataset 불러오기

from sklearn.datasets import make_moons

from sklearn.model_selection import train_test_split

X,y = make_moons(n_samples=100, noise=0.15)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting = "hard"
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [ ]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.9
RandomForestClassifier 1.0
SVC 1.0
VotingClassifier 1.0


투표 기반 분류기  
: 여러 개의 분류기를 만들어서 해당 분류기들의 결과를 투표해 최종 예측 결과를 내는 모델   
- 하드 보팅   
 : 다수결 투표, 각 분류기가 뭘 골랐는지만 고려
- 소프트 보팅
 : 다수결인데, 각 분류기가 해당 클래스에 대한 점수를 얼마나 줬는지도 같이 고려함.


# 7.2 배깅과 페이스팅

## 7.2.1 사이킷런의 배깅과 페이스팅

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=50, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

배깅과 페이스팅의 한 종류의 분류기를 사용하고 여러 개의 데이터셋으로 기존의 데이터셋을 나눠 학습시킨 후 투표하는 방식  
- Bagging: 중복을 허용해서 서브 데이터셋 추출
- Pasting: 중복을 허용하지 않고 서브 데이터셋을 비복원 추출

## 7.2.2 oob 평가

In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators = 500,
    bootstrap = True, n_jobs = -1, oob_score = True)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.925

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

1.0

In [ ]:
bag_clf.oob_decision_function_

array([[0.        , 1.        ],
       [0.01015228, 0.98984772],
       [0.97461929, 0.02538071],
       [0.94915254, 0.05084746],
       [0.97206704, 0.02793296],
       [0.        , 1.        ],
       [0.97435897, 0.02564103],
       [0.375     , 0.625     ],
       [0.97938144, 0.02061856],
       [0.7539267 , 0.2460733 ],
       [0.69590643, 0.30409357],
       [0.83888889, 0.16111111],
       [0.96097561, 0.03902439],
       [0.93258427, 0.06741573],
       [1.        , 0.        ],
       [0.90532544, 0.09467456],
       [0.        , 1.        ],
       [0.30150754, 0.69849246],
       [0.92857143, 0.07142857],
       [0.98295455, 0.01704545],
       [0.22885572, 0.77114428],
       [0.99421965, 0.00578035],
       [0.        , 1.        ],
       [0.28729282, 0.71270718],
       [0.48924731, 0.51075269],
       [0.75862069, 0.24137931],
       [0.95979899, 0.04020101],
       [0.82777778, 0.17222222],
       [0.01086957, 0.98913043],
       [0.98333333, 0.01666667],
       [0.

배깅 사용 시 사용되는 훈련 샘플은 전체 데이터의 63%정도,  
**oob(out of bag) = 선택되지 않은 훈련 세트의 나머지**   
예측기가 훈련되는 동안에는 oob 샘플을 사용하지 않기 때문에 별도의 검증 세틀르 사용하지 않고 oob 샘플로 평가 가능  
-> `oob_score = True` 로 지정

# 7.3 랜덤 패치와 랜덤 서브 스페이스

# 7.4 랜덤 포레스트

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators = 500, max_leaf_nodes = 16, n_jobs = -1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features = "auto", max_leaf_nodes = 16),
    n_estimators = 500, max_samples = 1.0, bootstrap = True, n_jobs = -1)

## 7.4.1 엑스트라 트리

: 극단적으로 무작위한 트리의 랜덤 포레스트 (= extremly randomized trees)  
트리를 더욱 무작위하게 만들기 위해 최적의 임곗값을 찾는 대신 후보특성을 사용해 무작위로 분할한 다음 그중 최상의 분할을 선택  
-> `ExtraTreesClassifier`, `ExtraTreesRegressor`

## 7.4.2 특성 중요도

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators = 500, n_jobs = -1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.09693671649737372
sepal width (cm) 0.024612811609673598
petal length (cm) 0.4254942085810035
petal width (cm) 0.45295626331194927


# 7.5 부스팅

: 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법  

## 7.5.1 에이다 부스트

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth = 1), n_estimators = 200,
    algorithm = "SAMME.R", learning_rate = 0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

## 7.5.2 그레이디언트 부스팅

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth = 2)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2)

In [ ]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth = 2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2)

In [ ]:
y3 = y2 - tree_reg1.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth = 2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2)

In [ ]:
#y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth = 2, n_estimators = 3, learning_rate = 1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt = GradientBoostingRegressor(max_depth = 2, n_estimators = 120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth = 2, n_estimators = bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=114)

In [ ]:
gbrt = GradientBoostingRegressor(max_depth = 2, warm_start = True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break  #조기종료

In [ ]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

In [ ]:
xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train,
            eval_set = [(X_val, y_val)], early_stopping_rounds = 2)
y_pred = xgb_reg.predict(X_val)

TypeError: XGBModel.fit() got an unexpected keyword argument 'early_stopping_rounds'

https://stackoverflow.com/questions/78713048/xgbclassifier-fit-got-an-unexpected-keyword-argument-early-stopping-rounds  
모델에 입력해줘야 한다고 합니다. 따라서 아래와 같이 수정합니다.

In [ ]:
xgb_reg = xgboost.XGBRegressor(early_stopping_rounds = 2)
xgb_reg.fit(X_train, y_train,
            eval_set = [(X_val, y_val)])
y_pred = xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.43240
[1]	validation_0-rmse:0.37292
[2]	validation_0-rmse:0.34417
[3]	validation_0-rmse:0.33315
[4]	validation_0-rmse:0.31313
[5]	validation_0-rmse:0.29548
[6]	validation_0-rmse:0.28934
[7]	validation_0-rmse:0.28140
[8]	validation_0-rmse:0.28450


# 7.6 스태킹

: 앙상블에 속한 모든 예측기의 예측을 예측하고, 그 예측값을 마지막 예측기(= 블렌더)에 넣어 최종 예측 출력  
